# Multivariable linear regression 

__Assumptions__
1. We are not using any type of regularisation technique.
2. For formulas I used one-based indexing while in code we are using zero-based indexing

__Terminologies__
1. m represents no. of examples(rows) and n represents no. of features(columns).
2. $w_j$ is coefficient of jth feature($x_j$) and b is constant in the model(equation)
3. $\vec{w}$ vector represents all coefficients of the model.
4. $\vec{x}$ vector represents the single row with all values

In [45]:
import numpy as np
import math

## Model

$$ \hat{y} = f_{\vec{w}, b}(\vec{x}) =  \sum_{j=1}^{n} (w_jx_j) + b$$


$$ \text{or}$$

$$ \hat{y} = f_{\vec{w}, b}(\vec{x})  =\vec{w}. \vec{x} + b $$

here $n$ is no. of features \
$\vec{w}$ and $b$ are model parameters

In [46]:
def predict(X, w, b):
    '''
    Computes the output of the model when given model parameters and inputs
    
    Args:
    
    X (ndarray(m, n)): m examples with n features
    w (ndarray(n,)): model parameters(n weights for n features).
    b (scalar): model parameter.

    Returns:
        f_wb (ndarray(m, )): m predictions for m input examples
    '''

    f_wb = np.dot(X, w.T) + b

    return f_wb

# test
X_temp = np.array([[1,2,3,4], [1,2,3,4]])
w_temp = np.array([1,2,3,4])
b_temp = 0

predict(X_temp, w_temp, b_temp) #array([30, 30])

array([30, 30])

# Cost function

For training purpose, _Squared error_ is used

$$ J(\vec{w}, b) = \frac{1}{2m} \sum_{i=1}^{m}(\hat{y}^{(i)} - y^{(i)})^2 $$
$$ \text{or} $$
$$ J(\vec{w}, b) = \frac{1}{2m} \sum_{i=1}^{m}(f_{\vec{w}, b}(\vec{x}^{(i)})- y^{(i)})^2 $$

$$ \text{or} $$

$$ J(\vec{w}, b) =  \frac{1}{2m} \sum_{i=1}^{m}(\vec{w}.\vec{x}^{(i)} + b - y^{(i)})^2$$

where $m$ is number of training exapmles.\
$i$ represents the ith example not __power__

In [47]:
def compute_squared_error(X, y, w, b):
    '''
    Returns squared error when given input, output and model parameters

    Args:
        X (ndarray(m, n)): m examples and n features
        y (ndarray(m)): m outputs for m examples
        w (ndarray(n, )): model parameter
        b (scalar): model parameter

    Returns:
        cost (scalar): Squared error
    '''
    m = X.shape[0]  #  m is no of examples in X
    f_wb = np.dot(X, w.T) + b
    error = (f_wb - y)**2
    cost = np.sum(error) / (2*m)

    return cost

# test
X_temp = np.array(
    [
        [1,2,3,4], 
        [1,2,3,4]
    ])
y_temp = [1,1]
w_temp = np.array([1,2,3,4])
b_temp = 0

compute_squared_error(X_temp, y_temp, w_temp, b_temp) # 420.5

420.5

## Gradient
The gradient is used to know the slope of the cost function at at some $\vec{w} \text{ and } b$. It tell us in which direction to move the parameter such that cost fucntion become minimal.

$$ \frac{\partial{J(\vec{w}, b)}}{\partial{w_j}} = \frac{1}{m} \sum_{i=1}^{m}(\hat{y}^{(i)} - y^{(i)})x_j$$
$$ \frac{\partial{J(\vec{w}, b)}}{\partial{b}} = \frac{1}{m} \sum_{i=1}^{m}(\hat{y}^{(i)} - y^{(i)})$$

For each paramerter the gradient is determined 

here $\frac{\partial{J(\vec{w}, b)}}{\partial{w_j}}$ is gradient wrt weight of jth feature  

$\frac{\partial{J(\vec{w}, b)}}{\partial{b}}$ is gradient wrt bias(constant without feature)

In [48]:
def compute_gradient(X, y, w, b):
    '''
    returns partial differenciation of cost function w.r.t all model parameters(w, b)

    Args:
        X (ndarray(m, n))
        y (ndarray(m))
        w (ndarray(m))
    '''
    m = X.shape[0]
    
    f_wb = np.dot(X,w) + b
    error = f_wb - y
    dJ_dw = np.dot(X.T, error) / m
    dJ_db = np.sum(error) / m
    
    return dJ_dw, dJ_db

# test
X_temp = np.array(
    [
        [1,2,3,4], 
        [1,2,3,4]
    ])
y_temp = [1,1]
w_temp = np.array([1,2,3,4])
b_temp = 0
compute_gradient(X_temp, y_temp, w_temp, b_temp) # (array([ 29.,  58.,  87., 116.]), 29.0)

(array([ 29.,  58.,  87., 116.]), 29.0)

## Gradient descent
- Gradient descent is also called _Update rule_
- It tells in which direction to modify parameters to make squared error(cost) of the model minimal.
 
$$ w_j = w_j - \alpha \frac{\partial{J(\vec{w}, b)}}{\partial{w_j}} $$

$$ b = b - \alpha \frac{\partial{J(\vec{w}, b)}}{\partial{b}} $$

where $j$ belongs from 1 to n \
$\alpha$ is learning rate. It controls to magnitude with which model parameters should update. 

In [49]:
def gradient_descent(X, y, iterations=1000, *, alpha=0.01, w_in=None, b_in=None):
    '''
    Returns final model parameters(w, b) after training

    Args:
        X (ndarray(m, n))
        y (ndarray(m,))
        iterations (scalar): no of times gardient decesnt is applied to update w and b to minise cost
        alpha (scalar): It represents learning rate
        w_in (ndarray(n,)): initial parameter value(optional)
        b_in (scalar): initial parameter value(optiona)

    Return:
        w (ndarray(n,)): model paramerter after training
        b (scalar): model paramerter after training
    '''
    m, n = X.shape
    
    w = w_in if w_in != None else np.zeros(n)
    b = b_in if b_in != None else 0.0
    
    for _ in range(iterations):
        dJ_dw, dJ_db = compute_gradient(X, y, w, b)
        if np.isnan(w).any() or np.isnan(dJ_dw).any():
            print("NaN encountered")
            print(f"w: {w}")
            print(f"dJ_dw: {dJ_dw}")
        if np.isinf(w).any() or np.isinf(dJ_dw).any():
            print("Infinity encountered")
            print(f"w: {w}")
            print(f"dJ_dw: {dJ_dw}")

        w = w - alpha*dJ_dw
        b = b - alpha*dJ_db
        
    return w, b

## Let us test gradient_descent

In [50]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X, y = fetch_california_housing(return_X_y=True)
X_norm = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, random_state=42)
w_final, b_final = gradient_descent(X_train, y_train)

print(f"Squared error for training data: {compute_squared_error(X_train, y_train, w_final, b_final)}")
print(f"Squared error for test data: {compute_squared_error(X_test, y_test, w_final, b_final)}")

Squared error for training data: 0.2756857061752969
Squared error for test data: 0.27755306514461875


#### fit() function is same as gradient_descent(), only differnce is it don't dependent on other fucntion(such as compute_gradient())

In [51]:
def fit(X, y, iterations=10000, *, alpha=0.01, w_in=None, b_in=None):
    m, n = X.shape
    
    w = w_in if w_in else np.zeros(n)
    b = b_in if b_in else 0.0

    for _ in range(iterations):
        f_wb = np.dot(X, w.T) + b
        error = f_wb - y

        dJ_dw = np.matmul(X.T, error) / m
        dJ_db = np.sum(error) / m

        w = w - alpha*dJ_dw
        b = b - alpha*dJ_db

    return w, b

# test
w_final, b_final = fit(X_train, y_train)

print(f"Squared error for training data: {compute_squared_error(X_train, y_train, w_final, b_final)}")
print(f"Squared error for test data: {compute_squared_error(X_test, y_test, w_final, b_final)}")

Squared error for training data: 0.260276475430372
Squared error for test data: 0.2706067903150232


In [54]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)

w_final, b_final = reg.coef_, reg.intercept_

print(f"Expected Squared error for training data: {compute_squared_error(X_train, y_train, w_final, b_final)}")
print(f"Expecteed Squared error for test data: {compute_squared_error(X_test, y_test, w_final, b_final)}")

Expected Squared error for training data: 0.26027610818225644
Expecteed Squared error for test data: 0.27056437392353444
